Constructing the company dataset, joining moneyball 1.1 successful and failed companies + scraping linkedin

In [1]:
import pandas as pd
import openpyxl
import pickle
import sys
import json
from datetime import datetime

In [2]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Successful Companies

In [3]:
s_1 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_success.xlsx", sheet_name = 'Company')
s_2 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_success.xlsx", sheet_name = 'Long company descriptions')
s_3 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_success.xlsx", sheet_name = 'Funding rounds')
s_4 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_success.xlsx", sheet_name = 'Founder Linkedin URLs')
weave_successful = []

In [4]:
df_s = pd.merge(s_1, s_2, on = ['org_uuid', 'org_name']).drop(columns = 'short_description')
df_s = pd.merge(df_s, s_3, on = ['org_uuid', 'org_name'])
df_s = pd.merge(df_s, s_4, on = ['org_uuid', 'org_name'])
df_s['founded_on'] = [c.strftime("%Y") for c in df_s['founded_on']]
df_s.dropna(inplace = True)
df_s = df_s[df_s['raised_amount_usd'] > 0]

In [5]:
df_inv_agg_s = df_s.groupby(['org_name', 'org_uuid'])[['investor_count', 'raised_amount_usd']].sum().reset_index()
df_inv_agg_s['investor_number_normalised']=(df_inv_agg_s['investor_count']-df_inv_agg_s['investor_count'].mean())/df_inv_agg_s['investor_count'].std()
df_inv_agg_s['raised_amount_usd_normalised']=(df_inv_agg_s['raised_amount_usd']-df_inv_agg_s['raised_amount_usd'].mean())/df_inv_agg_s['raised_amount_usd'].std()

In [6]:
investor_number = []
for c in df_inv_agg_s['investor_number_normalised']:
    if c <= 0.1:
        investor_number.append('low')
    elif c >= 0.1 and c < 1.5:
        investor_number.append('mid')
    elif c >= 1.5:
        investor_number.append('high')

df_inv_agg_s['investor_number'] = investor_number
#df_inv_agg_s.drop(columns = ['investor_count','investor_number_normalised'], inplace = True) 

In [7]:
raised_amount = []
for c in df_inv_agg_s['raised_amount_usd_normalised']:
    if c <= 0.1:
        raised_amount.append('low')
    elif c >= 0.1 and c < 2:
        raised_amount.append('mid')
    elif c >= 2:
        raised_amount.append('high')
df_inv_agg_s['investment_size'] = raised_amount


In [8]:
df_s = df_s.merge(df_inv_agg_s[['org_name', 'org_uuid','investor_number_normalised', 'raised_amount_usd_normalised']], how = 'left', on = ['org_name', 'org_uuid'])
# df_s.drop(columns = 'category_list', inplace = True)
# df_s.rename(columns = {'category_groups_list': 'category_list'}, inplace = True)
# df_s.to_excel(r'/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/successful_comp.xlsx', index=False)


In [9]:
# GNN features generation
df_s_gnn_inv_Agg = df_s.groupby(['org_name', 'org_uuid'])[['investor_count', 'raised_amount_usd']].sum().reset_index()
# df_s_gnn_inv_Agg.rename(columns = {'investor_count': 'total_investor_number',
#                                    'raised_amount_usd':'total_raised_amount'}, inplace = True)
df_s_gnn = df_s.merge(df_s_gnn_inv_Agg, how = 'left', on = ['org_name', 'org_uuid'])
df_s_gnn = df_s_gnn[['org_uuid', 'org_name', 'founded_on','country_code', 'city',
       'investment_type', 'investors_names', 'investors_uuids', 'round_created_at',
       'founder_linkedin_url', 'investor_number_normalised', 'raised_amount_usd_normalised']]

In [10]:
df_s_gnn_date = df_s_gnn.groupby(['org_name', 'org_uuid'])['round_created_at'].apply(
    lambda x:max(list(datetime.strptime(y,'%Y-%m-%d %H:%M:%S.000000 UTC') for y in x))
).reset_index()

In [11]:
df_s_gnn['round_created_at'] = df_s_gnn['round_created_at'].apply(
    lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S.000000 UTC')
    )

In [12]:
df_s_gnn = pd.merge(df_s_gnn, df_s_gnn_date, how='right', on=['org_name', 'org_uuid','round_created_at'])
# multiple different founders left for each company

Failing Companies

In [23]:
fail_1 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_fail.xlsx", sheet_name = 'Company')
fail_2 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_fail.xlsx", sheet_name = 'Long company descriptions')
fail_3 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_fail.xlsx", sheet_name = 'Funding rounds')
fail_4 = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/moneyball_fail.xlsx", sheet_name = 'Founder Linkedin URLs')
weave_fail = []


In [24]:
df_f = pd.merge(fail_1, fail_2, on = ['org_uuid', 'org_name']).drop(columns = 'short_description')
df_f = pd.merge(df_f, fail_3, on = ['org_uuid', 'org_name'])
df_f = pd.merge(df_f, fail_4, on = ['org_uuid', 'org_name'])
df_f['founded_on'] = [c.strftime("%Y") for c in df_f['founded_on']]
df_f = df_f[df_f['raised_amount_usd'] > 0]
df_f.dropna(inplace = True)

In [25]:
df_inv_agg_f = df_f.groupby(['org_name', 'org_uuid'])[['investor_count', 'raised_amount_usd']].sum().reset_index()
df_inv_agg_f['investor_number_normalised']=(df_inv_agg_f['investor_count']-df_inv_agg_f['investor_count'].mean())/df_inv_agg_f['investor_count'].std()
df_inv_agg_f['raised_amount_usd_normalised']=(df_inv_agg_f['raised_amount_usd']-df_inv_agg_f['raised_amount_usd'].mean())/df_inv_agg_f['raised_amount_usd'].std()

In [26]:
investor_number = []
for c in df_inv_agg_f['investor_number_normalised']:
    if c <= 0.1:
        investor_number.append('low')
    elif c >= 0.1 and c < 1.5:
        investor_number.append('mid')
    elif c >= 1.5:
        investor_number.append('high')


df_inv_agg_f['investor_number'] = investor_number
# df_inv_agg_f.drop(columns = ['investor_count','investor_number_normalised'], inplace = True)

raised_amount = []
for c in df_inv_agg_f['raised_amount_usd_normalised']:
    if c <= 0.1:
        raised_amount.append('low')
    elif c >= 0.1 and c < 2:
        raised_amount.append('mid')
    elif c >= 2:
        raised_amount.append('high')


df_inv_agg_f['investment_size'] = raised_amount
# df_inv_agg_f.drop(columns = ['raised_amount_usd_normalised', 'raised_amount_usd'], inplace = True)




In [27]:
df_f = df_f.merge(df_inv_agg_f[['org_name', 'org_uuid','investor_number_normalised', 'raised_amount_usd_normalised']], how = 'left', on = ['org_name', 'org_uuid'])
# df_f.rename(columns = {'category_groups_list': 'category'}, inplace = True)
# df_f = df_f[['org_uuid', 'org_name', 'founded_on',
#        'category_list','country_code', 'city', 'long_description',
#        'investment_type', 'investors_names', 'investors_uuids',
#        'founder_linkedin_url', 'investor_number', 'investment_size']]
# df_f.to_excel(r'/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/failed_comp.xlsx', index=False)


In [28]:
# GNN features generation
df_f_gnn_inv_Agg = df_f.groupby(['org_name', 'org_uuid'])[['investor_count', 'raised_amount_usd']].sum().reset_index()
df_f_gnn_inv_Agg.rename(columns = {'investor_count': 'total_investor_number',
                                   'raised_amount_usd':'total_raised_amount'}, inplace = True)
df_f_gnn = df_f.merge(df_f_gnn_inv_Agg, how = 'left', on = ['org_name', 'org_uuid'])
df_f_gnn = df_f_gnn[['org_uuid', 'org_name', 'founded_on','country_code', 'city',
       'investment_type', 'investors_names', 'investors_uuids', 'round_created_at',
       'founder_linkedin_url', 'investor_number_normalised', 'raised_amount_usd_normalised']]

In [29]:
df_f_gnn_date = df_f_gnn.groupby(['org_name', 'org_uuid'])['round_created_at'].apply(
    lambda x:max(list(datetime.strptime(y,'%Y-%m-%d %H:%M:%S.000000 UTC') for y in x))
).reset_index()

In [30]:
df_f_gnn['round_created_at'] = df_f_gnn['round_created_at'].apply(
    lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S.000000 UTC')
    )

In [31]:
df_f_gnn = pd.merge(df_f_gnn, df_f_gnn_date, how='right', on=['org_name', 'org_uuid','round_created_at'])
# multiple different founders left for each company

Investors

In [32]:
inv = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/investor_profiles.csv')
inv.dropna(inplace = True)
inv['investment_count_normalised']=(inv['investment_count']-inv['investment_count'].mean())/inv['investment_count'].std()

In [33]:
investment_count = []
for c in inv['investment_count_normalised']:
    if c <= 0.1:
        investment_count.append('D')
    elif c >= 0.1 and c < 1:
        investment_count.append('C')
    elif c >= 1 and c < 10:
        investment_count.append('B')
    elif c >= 10:
        investment_count.append('A')

inv['investment_count'] = investment_count
inv.drop(columns = 'investment_count_normalised', inplace = True)
# inv.to_excel(r'/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/investor-df.xlsx', index=False)


In [34]:
df_i = pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/investor-df.xlsx")
df_f = df_f_gnn
df_s = df_s_gnn

In [35]:
# Add Labels: 1 for success, 0 for fail
df_s['labels'] = 1
df_f['labels'] = 0

Find Founders Info !!!!!!!!

In [36]:
# Load founders profiles
success_enriched_linkedin_profiles = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/success_enriched_linkedin_profiles.csv', index_col='linkedin_url')
fail_enriched_linkedin_profiles = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/fail_enriched_linkedin_profiles.csv', index_col='linkedin_url')

In [50]:
test_json = json.loads(success_enriched_linkedin_profiles.loc['https://www.linkedin.com/in/martin-pule-3a597978'][0])
test_json['data'][0]['educations']

[{'institution': {'summary': 'University in Botswana',
   'image': 'https://kg.diffbot.com/image/api/get?fetch=yes&url=g%3Cj7P0St%7D9u.LnOv%7BI%7C%40g7.pBYINs%3Ff%7CGZ7jr-tGZhWf%3DK%2FOvy%5Bp%3CSKbs8i%7CEy%3FiGQn9dyW%3D8U0Um3dv9T',
   'types': ['Organization', 'Corporation', 'Company'],
   'name': 'University of Botswana',
   'diffbotUri': 'http://diffbot.com/entity/EXKYl8_kyOSKVfSG-T7NxiA',
   'targetDiffbotUri': 'http://diffbot.com/entity/EXKYl8_kyOSKVfSG-T7NxiA',
   'targetDiffbotId': 'EXKYl8_kyOSKVfSG-T7NxiA',
   'surfaceForm': 'University of Botswana',
   'type': 'Organization'},
  'isCurrent': False,
  'from': {'str': 'd2011-XX-XX', 'precision': 1, 'timestamp': 1293840000000},
  'to': {'str': 'd2017-XX-XX', 'precision': 1, 'timestamp': 1483228800000}},
 {'institution': {'recordId': 'EEzIb4GfSPnmCCr36BaLMFA@27',
   'name': 'University College Dublin',
   'surfaceForm': 'University College Dublin',
   'position': 'personEducationInstitution',
   'type': 'Organization'},
  'degree':

In [51]:
df_s1 = df_s[['org_uuid', 'org_name', 'founded_on', 'country_code', 'city',
       'investment_type', 'investors_names', 'investors_uuids',
       'round_created_at', 'founder_linkedin_url', 'investor_number_normalised',
       'raised_amount_usd_normalised', 'labels']]
df_f1 = df_f[['org_uuid', 'org_name', 'founded_on', 'country_code', 'city',
       'investment_type', 'investors_names', 'investors_uuids',
       'round_created_at', 'founder_linkedin_url', 'investor_number_normalised',
       'raised_amount_usd_normalised', 'labels']]

In [52]:
# Load founder info 
def insert_new_cols(df):
  df.insert(0,'gender',pd.NA)
  df.insert(0,'age',pd.NA)
  df.insert(0,'past_roles',pd.NA)
  df.insert(0,'past_companies',pd.NA)
  df.insert(0,'education',pd.NA)
  df.insert(0,'degrees',pd.NA)

In [53]:
# function to update a column
def update_df(df,column_change,column_id,id,new_value):
  for index,element in enumerate(df[column_id]):
    if element == id:
      df.loc[index,column_change] = new_value

In [30]:
# # function to update a column
# def update_df_list(df,column_change,column_id,id,new_value):
#   df[column_change] = df[column_change].astype(object)
#   for index,element in enumerate(df[column_id]):
#     if element == id:    
#       df.at[index,column_change] = new_value

In [54]:
# function to ensure unix timestamp does not exceed current time and in 10 digits if not 9
def check_timestamp(unix_timestamp):
  if unix_timestamp < 0:
    unix_timestamp = -unix_timestamp
  # ensure time stamp is 10 digits
  while unix_timestamp > 253402300799:
    unix_timestamp = unix_timestamp/10
  while datetime.now()<datetime.fromtimestamp(unix_timestamp):
    unix_timestamp = unix_timestamp/10
  return unix_timestamp

In [32]:
# from pandas.core import series
# import math

# # Update data frame
# def update_all_cols(df,enriched_linkedin_profiles):
#   no_noEducation = 0
#   no_noDegree = 0
#   no_noEmployment = 0
#   for url in df['founder_linkedin_url']:
#     # check founder_url is not empty
#     if isinstance(url,str):
#       # Check url does not repeat: if repeat a series is returned
#       if isinstance(enriched_linkedin_profiles.loc[url,'json_string'],pd.Series):
#         profile_json = enriched_linkedin_profiles.loc[url,'json_string'][0]
#       else:
#         profile_json = enriched_linkedin_profiles.loc[url,'json_string']
      
#       # check whether profile_json is NA
#       if not pd.isna(profile_json):
#         profile_json = json.loads(profile_json)

#         # check 'data' exists
#         if 'data' in profile_json:

#           # check 'data' is not empty
#           if len(profile_json['data']) != 0:

#             # check 'age' exist
#             if 'age' in profile_json['data'][0]:
#               age = profile_json['data'][0]['age']
#               update_df(df,'age','founder_linkedin_url',url,age) 
#               # df.loc[url,'age'] = age
#             else:
#               pass
#               # print('age not in')
#               # print(url, profile_json['data'])
            
#             # check 'gender' exist: note that 'gender' might have different keys
#             if 'gender' in profile_json['data'][0]:
#               if 'normalizedValue' in profile_json['data'][0]['gender']:
#                 gender = profile_json['data'][0]['gender']['normalizedValue']
#                 update_df(df,'gender','founder_linkedin_url',url,gender)
#                 # df.loc[url,'gender'] = gender
#               elif 'str' in profile_json['data'][0]['gender']:
#                 gender = profile_json['data'][0]['gender']['str']
#                 update_df(df,'gender','founder_linkedin_url',url,gender)
#                 # df.loc[url,'gender'] = gender
#               else:
#                 print(profile_json['data'][0]['gender'])
              
#             # check 'education' exist
#             if 'educations' in profile_json['data'][0]:
#               educations = []
#               degrees = []
#               for education in profile_json['data'][0]['educations']:
#                 if 'institution'in education:
#                   if 'degree' in education:
#                     if ('name' in education['institution']) and ('name' in education['degree']):
#                       educations.append(education['institution']['name'])
#                       degrees.append(education['degree']['name'])
#                   else:
#                     if 'name' in education['institution']:
#                       educations.append(education['institution']['name'])
#               if educations:
#                 update_df_list(df,'education','founder_linkedin_url',url,educations)
#               if degrees:
#                 update_df_list(df,'degrees','founder_linkedin_url',url,degrees)
#               else:
#                 no_noDegree = no_noDegree + 1
#             else:
#               # print("'educations' don't exist")
#               # print(f'this dude does not have education data: {url}')
#               no_noEducation = no_noEducation + 1
            

#             # check past companies exist
#             if 'employments' in profile_json['data'][0]:
#               employments = []
#               roles = []
#               for employment in profile_json['data'][0]['employments']:
#                 if 'employer' in employment:
#                   if 'name' in employment['employer']:
#                     employments.append(employment['employer']['name'])
#                 else:
#                   pass
#                   # print(employment)
#                   # print(f'This dude has no employer: {url}')
#                   # break
#                 if 'title' in employment:
#                   roles.append(employment['title'])

#               if employments:
#                 update_df_list(df,'past_companies','founder_linkedin_url',url,employments)
#               if roles:
#                 update_df_list(df,'past_roles','founder_linkedin_url',url,roles)

#             else:
#               # pass
#               print(profile_json['data'][0])
#               print(f'No employments for this man: {url}')
#               no_noEmployments = no_noEmployment + 1

#           else:
#             pass
#             # print(url, profile_json['data'])
#             # print("'data' is a empty list")  
#         else:
#           pass
#           # print("'data' not in profile_json")
#       else:
#         pass
#         # print('profile_json is NA')
#       # try:
#       #   profile_json = json.loads(profile_json)
#       # except:
#       #   print(pd.isna(enriched_linkedin_profiles.loc[url,'json_string']))
#       #   print(url, profile_json)
#       #   print('failed to load json')
#     else:
#       print('url not a string')

In [33]:
# insert_new_cols(df_s1)
# update_all_cols(df_s1,success_enriched_linkedin_profiles)

In [55]:
from pandas.core import series
import math

# Update data frame (simplified): only include latest position/company/education
def update_all_cols_simplified(df,enriched_linkedin_profiles):
  no_noEducation = 0
  no_noDegree = 0
  no_noEmployment = 0
  for url in df['founder_linkedin_url']:
    # check founder_url is not empty
    if isinstance(url,str):
      # Check url does not repeat: if repeat a series is returned
      if isinstance(enriched_linkedin_profiles.loc[url,'json_string'],pd.Series):
        profile_json = enriched_linkedin_profiles.loc[url,'json_string'][0]
      else:
        profile_json = enriched_linkedin_profiles.loc[url,'json_string']
      
      # check whether profile_json is NA
      if not pd.isna(profile_json):
        profile_json = json.loads(profile_json)

        # check 'data' exists
        if 'data' in profile_json:

          # check 'data' is not empty
          if len(profile_json['data']) != 0:

            # check 'age' exist
            if 'age' in profile_json['data'][0]:
              age = profile_json['data'][0]['age']
              update_df(df,'age','founder_linkedin_url',url,age) 
              # df.loc[url,'age'] = age
            else:
              pass
              # print('age not in')
              # print(url, profile_json['data'])
            
            # check 'gender' exist: note that 'gender' might have different keys
            if 'gender' in profile_json['data'][0]:
              if 'normalizedValue' in profile_json['data'][0]['gender']:
                gender = profile_json['data'][0]['gender']['normalizedValue']
                update_df(df,'gender','founder_linkedin_url',url,gender)
                # df.loc[url,'gender'] = gender
              elif 'str' in profile_json['data'][0]['gender']:
                gender = profile_json['data'][0]['gender']['str']
                update_df(df,'gender','founder_linkedin_url',url,gender)
                # df.loc[url,'gender'] = gender
              else:
                print(profile_json['data'][0]['gender'])
              
            # check 'education' exist
            if 'educations' in profile_json['data'][0]:
              educations = {}
              degrees = {}
              for education in profile_json['data'][0]['educations']:
                if ('institution' in education) and ('from' in education):
                  if 'degree' in education:
                    if ('name' in education['institution']) and ('name' in education['degree']) and ('timestamp' in education['from']):
                      unix_timestamp = education['from']['timestamp']
                      if unix_timestamp == 0:
                        continue
                      unix_timestamp = check_timestamp(unix_timestamp)
                      from_timestamp = datetime.fromtimestamp(unix_timestamp)
                      educations[from_timestamp] = education['institution']['name']
                      degrees[from_timestamp] = education['degree']['name']
                  else:
                    pass
                    # if 'name' in education['institution']:
                    #   educations.append(education['institution']['name'])
              if educations:
                latest_education = educations[max(list(from_time for from_time in educations))]
                update_df(df,'education','founder_linkedin_url',url,latest_education)
              if degrees:
                latest_degree = degrees[max(list(from_time for from_time in degrees))]
                update_df(df,'degrees','founder_linkedin_url',url,latest_degree)
              else:
                no_noDegree = no_noDegree + 1
            else:
              # print("'educations' don't exist")
              # print(f'this dude does not have education data: {url}')
              no_noEducation = no_noEducation + 1
            

            # check past companies exist
            if 'employments' in profile_json['data'][0]:
              employments = {}
              roles = {}
              for employment in profile_json['data'][0]['employments']:
                if ('employer' in employment) and ('from' in employment) and ("isCurrent" in employment):
                  if employment["isCurrent"] == True:
                    continue
                  else:
                    if 'name' in employment['employer']:
                      unix_timestamp = employment['from']['timestamp']
                      if unix_timestamp == 0:
                        continue
                      unix_timestamp = check_timestamp(unix_timestamp)
                      from_timestamp = datetime.fromtimestamp(unix_timestamp)
                      employments[from_timestamp] = employment['employer']['name']
                else:
                  pass
                  # print(employment)
                  # print(f'This dude has no employer: {url}')
                  # break
                if ('title' in employment) and ('from' in employment):
                  unix_timestamp = employment['from']['timestamp']
                  if unix_timestamp == 0:
                    continue
                  unix_timestamp = check_timestamp(unix_timestamp)
                  from_timestamp = datetime.fromtimestamp(unix_timestamp)
                  roles[from_timestamp] = employment['title']

              if employments:
                latest_employment = employments[max(list(from_time for from_time in employments))]
                update_df(df,'past_companies','founder_linkedin_url',url,latest_employment)
              if roles:
                latest_role = roles[max(list(from_time for from_time in roles))]
                update_df(df,'past_roles','founder_linkedin_url',url,latest_role)

            else:
              # pass
              print(profile_json['data'][0])
              print(f'No employments for this man: {url}')
              no_noEmployments = no_noEmployment + 1

          else:
            pass
            # print(url, profile_json['data'])
            # print("'data' is a empty list")  
        else:
          pass
          # print("'data' not in profile_json")
      else:
        pass
        # print('profile_json is NA')
      # try:
      #   profile_json = json.loads(profile_json)
      # except:
      #   print(pd.isna(enriched_linkedin_profiles.loc[url,'json_string']))
      #   print(url, profile_json)
      #   print('failed to load json')
    else:
      print('url not a string')

In [56]:
insert_new_cols(df_s1)
update_all_cols_simplified(df_s1,success_enriched_linkedin_profiles)

In [57]:
insert_new_cols(df_f1)
update_all_cols_simplified(df_f1,fail_enriched_linkedin_profiles)

Combine companies together

In [58]:
# Combine companies together
concat_companies = pd.concat([df_s1,df_f1],ignore_index=True)
concat_companies.dropna(inplace=True)
concat_companies.reset_index(inplace=True)
concat_companies.drop(columns='index',inplace=True)
# concat_companies.to_excel(r'/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/concat_companies.xlsx', index=False)

In [59]:
concat_companies['age'] = concat_companies['age'].astype(int)
reduced_concat_companies = concat_companies.groupby(['org_name', 'org_uuid']).apply(
    lambda g: g.nlargest(1,'age')
)
reduced_concat_companies.index = reduced_concat_companies.index.get_level_values(2)
reduced_concat_companies.sort_index(inplace=True)
reduced_concat_companies.reset_index(inplace=True,drop=True)

Create OpenAI embeddings

In [61]:
!pip install openai
import openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.4 MB/s eta 0:00:00


In [65]:
# openai.api_key = <API-KEY>

In [66]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [ ]:
reduced_concat_companies['education_embedding'] = reduced_concat_companies.education.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

In [73]:
reduced_concat_companies['degree_embedding'] = reduced_concat_companies.degrees.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

In [76]:
reduced_concat_companies['role_embedding'] = reduced_concat_companies.past_roles.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

In [79]:
reduced_concat_companies['company_embedding'] = reduced_concat_companies.past_companies.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

In [82]:
reduced_concat_companies.to_csv('/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/feature_vectors.csv', index=False)

In [90]:
reduced_concat_companies['age_normalised'] =  ( reduced_concat_companies['age'] - reduced_concat_companies['age'].mean()) / reduced_concat_companies['age'].std()


In [103]:
reduced_concat_companies['feature_vector'] = reduced_concat_companies['education_embedding'] + reduced_concat_companies['degree_embedding'] + reduced_concat_companies['company_embedding'] + reduced_concat_companies['role_embedding']


In [119]:
concat_companies_features = reduced_concat_companies

for index in range(len(concat_companies_features)):
  concat_companies_features.iloc[index]['feature_vector'].append(concat_companies_features.iloc[index]['age_normalised'])
  concat_companies_features.iloc[index]['feature_vector'].append(concat_companies_features.iloc[index]['investor_number_normalised'])
  concat_companies_features.iloc[index]['feature_vector'].append(concat_companies_features.iloc[index]['raised_amount_usd_normalised'])

In [131]:
len(concat_companies_features.iloc[0]['feature_vector'])

6147

In [128]:
concat_companies_features.to_csv('/content/gdrive/MyDrive/Colab Notebooks/weave/Datasets/feature_vectors_rescaled.csv', index=False)

Create edge data

In [ ]:
# create a dictionary with investor_uuid as key and a list of org_uuid as value
# !!!! assume multiple founders for each company
edges_base = {}

for org_index,org in enumerate(concat_companies['org_uuid']):
  # get a list of investor uuids for each company
  investors = concat_companies.loc[org_index,'investors_uuids']
  investors = investors.split(',')

  # put org_uuid for each investors_uuids
  for investor in investors:
    if edges_base.get(investor) == None: 
      edges_base[investor] = []
    if org not in edges_base[investor]:
      edges_base[investor].append(org)
    
    

print(f"The number of investors is {len(edges_base)}.")


In [ ]:
# with open('/content/gdrive/MyDrive/Colab Notebooks/weave/KnowledgeBase/edges_base.pickle', 'wb') as file:
#   pickle.dump(edges_base, file)